In [4]:
import sys
import os
import pandas as pd
sys.path.append(os.path.abspath(os.path.join('..', 'src')))
%matplotlib inline

In [8]:
%load_ext autoreload
%autoreload 2
from data import ApiFetcher, DataPreparation
import api.server as server

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
server.main()

INFO:     Will watch for changes in these directories: ['d:\\Projekty\\ML\\model\\notebooks']
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
INFO:     Started reloader process [7984] using StatReload


In [6]:
api = ApiFetcher(starting_year=2019, ending_year=2025)
df = api.get_dataframe('leaguegamelog')
df.head()

,home_fga,away_fga,home_fg_pct,away_fg_pct,home_fg3a,away_fg3a,home_fg3_pct,away_fg3_pct,home_oreb,away_oreb,...,away_blk,home_tov,away_tov,home_pf,away_pf,home_pts,away_pts,home_team_id,away_team_id,date
0,103,102,0.408,0.422,40,45,0.350,0.422,16,16,...,9,17,19,24,34,130,122,27,18,2019-10-22
1,81,85,0.519,0.435,31,33,0.355,0.394,11,9,...,7,14,15,25,24,112,102,12,13,2019-10-22
2,86,88,0.430,0.375,30,34,0.300,0.265,7,10,...,2,13,16,18,15,94,85,21,5,2019-10-23
3,76,93,0.461,0.398,36,41,0.306,0.268,5,11,...,3,20,17,22,32,108,100,6,29,2019-10-23
4,88,105,0.511,0.467,44,30,0.523,0.300,9,20,...,4,19,12,18,20,126,125,3,4,2019-10-23


In [7]:
dataPrepeparation = DataPreparation('leaguegamelog')
train_df, val_df, test_df = dataPrepeparation.data

In [10]:
features, label = train_df[0]
print(features.shape, label.shape)

torch.Size([10, 24]) torch.Size([1, 2])


In [9]:
df = api.get_dataframe('boxscoreadvanced')
df.head()

,game_id,home_team_id,away_team_id,home_minutes,home_est_off_rating,home_off_rating,home_est_def_rating,home_def_rating,home_est_net_rating,home_net_rating,...,away_efg_pct,away_ts_pct,away_usg_pct,away_est_usg_pct,away_est_pace,away_pace,away_pace_per40,away_possessions,away_pie,date
0,0021900001,27,18,265:00,565.2,113.0,544.6,108.9,20.6,4.1,...,0.515,0.551,1.0,1.000,20.56,102.79,85.66,112.0,0.469,2019-10-22
1,0021900002,12,13,240:00,577.3,115.5,525.8,105.2,51.5,10.3,...,0.512,0.541,1.0,0.964,19.40,97.00,80.83,97.0,0.420,2019-10-22
2,0021900005,21,5,240:00,470.0,94.0,425.0,85.0,45.0,9.0,...,0.426,0.449,1.0,0.982,20.00,100.00,83.33,100.0,0.454,2019-10-23
3,0021900009,6,29,240:00,519.2,103.8,480.8,96.2,38.5,7.7,...,0.457,0.495,1.0,0.983,20.80,104.00,86.67,104.0,0.432,2019-10-23
4,0021900003,3,4,240:00,600.0,120.0,601.0,120.2,-1.0,-0.2,...,0.510,0.545,1.0,0.976,20.90,104.50,87.08,104.0,0.494,2019-10-23


In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join('..', 'src')))


In [ ]:
from model.team_embeddings.embeddings_train import TeamEmbeddingsModel
from model.team_embeddings.embeddings_fetcher import EmbeddingFetcher
from data.api_fetcher import ApiFetcher

In [ ]:
api = ApiFetcher(starting_year=2014, ending_year=2025)
df = api.get_dataframe(numeric=False, date=True, time_coeff=False, ids=True)
model1 = TeamEmbeddingsModel(df)


c:\Users\szymo\AppData\Local\Programs\Python\Python313\Lib\site-packages\numpy\_core\fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [ ]:
test_loss, model = model1.train()


In [ ]:
home_embeddings = model.home_embedding.weight.detach().cpu().numpy()
away_embeddings = model.away_embedding.weight.detach().cpu().numpy()
print(embeddings_fetcher.get_home_embedding(team_id=5))

[-0.961085   -0.40299723 -0.6816333  -1.259706   -0.46907297  0.47658578
 -0.7354865  -0.22129872]
